In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pathlib, os, subprocess, json, datetime, shutil

REPO_URL   = "https://github.com/SzymonSmagowski/DeepLearningCourse.git"
BRANCH     = "main"
DATA_IN_DRIVE = "data/speech_commands_v0.01"

ROOT_DRIVE   = pathlib.Path("/content/drive/MyDrive")
DATA_SRC     = ROOT_DRIVE / DATA_IN_DRIVE
DATA_DST     = pathlib.Path("/content/data")                        # where the script expects it
REPO_DIR     = pathlib.Path("/content") / pathlib.Path(REPO_URL).stem / "project_2"
LOGS_DRIVE   = ROOT_DRIVE / "speech_runs"      # all logs live here

print("Repo dir :", REPO_DIR)
print("Data src :", DATA_SRC)
print("Data dst :", DATA_DST)
print("Logs dir :", LOGS_DRIVE)

In [ ]:
if not REPO_DIR.exists():
    !git clone -b "$BRANCH" "$REPO_URL" "$REPO_DIR"
else:
    %cd $REPO_DIR
    !git pull origin "$BRANCH"
    %cd -

In [ ]:
# -----------------------------------------------------------
# 📦  Get Speech-Commands into /content/data
#   1. If the *folder* already exists  → use it
#   2. Else if Drive has an extracted folder → symlink / copy
#   3. Else if Drive only has  *.tar.gz       → untar it once
# -----------------------------------------------------------
import tarfile, time, shutil, pathlib, os

DATA_TAR = ROOT_DRIVE / f"{DATA_IN_DRIVE}.tar.gz"   # e.g. …/speech_commands_v0.01.tar.gz
DATA_DST.mkdir(parents=True, exist_ok=True)         # creates /content/data *parent*

if (DATA_DST / "speech_commands_v0.01").exists():
    print("✓ dataset already present in /content — nothing to do")

elif (DATA_SRC).exists():
    # DATA_SRC points to the *folder* in Drive
    try:
        # fastest
        (DATA_DST / "speech_commands_v0.01").symlink_to(DATA_SRC, target_is_directory=True)
        print("🔗  Created symlink from Drive → /content")
    except Exception as e:
        print("Symlink failed (likely permissions); copying files …")
        shutil.copytree(DATA_SRC, DATA_DST / "speech_commands_v0.01")

elif DATA_TAR.exists():
    print("📦  Found compressed tarball in Drive – extracting …")
    t0 = time.time()
    with tarfile.open(DATA_TAR, "r:gz") as tf:
        tf.extractall(path=DATA_DST)
    print(f"✓ extracted in {time.time() - t0:.1f}s →",
          (DATA_DST / 'speech_commands_v0.01').as_posix())
else:
    raise FileNotFoundError(
        f"Dataset not found!\nLooked for:\n  • {DATA_SRC}\n  • {DATA_TAR}"
    )

In [ ]:
!pip install -q torch torchaudio librosa tqdm matplotlib seaborn scikit-learn

In [ ]:
!chmod +x $REPO_DIR/run_all.sh

In [ ]:
ts   = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
logf = LOGS_DRIVE / f"run_all_{ts}.log"
logf.parent.mkdir(exist_ok=True, parents=True)

cmd  = f"bash {REPO_DIR}/run_all.sh 2>&1 | tee {logf}"
print("⇢ running:", cmd)
exit_code = subprocess.call(cmd, shell=True)
print("\n=== finished, exit code:", exit_code, "===")
print("Full log saved to", logf)